経営管理でまとめている「＊＊＊＊年＊＊月度_承認口座数.xlsx」まとめるデータ

In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)Note: you may need to restart the kernel to use updated packages.



In [2]:
pip install -U openpyxl

  Using cached openpyxl-3.0.5-py2.py3-none-any.whl (242 kB)
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.4
    Uninstalling openpyxl-3.0.4:
      Successfully uninstalled openpyxl-3.0.4
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

#from pandasql import sqldf

In [4]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201108


In [5]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [6]:
#ディレクトリ指定
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202010_スコアカード")

新規入会数

In [287]:
filename = ["2020年10月度_承認口座数.xlsx"]
old_filename = ["2020年2月度_承認口座数.xlsx","2019年2月度_承認口座数.xlsx"]

In [303]:
li = []#空のデータフレーム作成

2020年度用

In [304]:
%%time

for f in filename:
    input_file = f
    input_book = pd.ExcelFile(data_dir/ f"{input_file}")
    #各シート名を取得
    input_sheet_name = input_book.sheet_names
    num_sheet = len(input_sheet_name)
    
    for i in range(1,num_sheet-3):#不要なシートが一つあるので読みこまない
        df = input_book.parse(input_sheet_name[i],header = None)

        if f[:5] == "2020年":
            df = df.iloc[[5,6,7,8,9,10,11,13,14],[2,22]]#該当行と列のみ抽出
            df = df.rename(columns={2:"card",22:"新規入会数"})

            #ブランド分け
            df.loc[df['card'].str.contains("VISA"), 'brand'] = 'VISA'
            df.loc[df['card'].str.contains("Master"), 'brand'] = 'Master'
            df.loc[df['card'] .str.contains("臨時"), 'brand'] = 'VISA'

            df.loc[df['card'].str.contains("グループ※"), 'card'] = '一般'
            df.loc[df['card'].str.contains("臨時"), 'card'] = '臨時カード'

            df['card'] = df['card'].str.replace("VISA",'')
            df['card'] = df['card'].str.replace("Master",'')
            
            #年月の列を作成
            df['filename'] = f            
            df['month'] = input_sheet_name[i]
            df['month'] = df['month'].astype(int)
            df['year'] = df['filename'].str[:4]
            df['year'] = df['year'].astype(int)
            df = df.drop(columns = {"filename"})

            li.append(df)
            
            #df_all = pd.concat(li, axis=0, ignore_index=True)

#df_all = df_all.reindex(columns = ['year','month','brand','card','新規入会数'])

Wall time: 313 ms


2019年度用

In [305]:
%%time
for f in old_filename:
    input_file = f
    input_book = pd.ExcelFile(data_dir/ f"{input_file}")
    #各シート名を取得
    input_sheet_name = input_book.sheet_names
    num_sheet = len(input_sheet_name)
    
    for i in range(1,num_sheet-3):#不要なシートが一つあるので読みこまない
        df = input_book.parse(input_sheet_name[i],header = None)
        
        if f[:8] == f[:8] == "2020年2月度":
            df = df.iloc[[4,5,6,7,8,9,11,12,15,16],[2,23]]#該当行と列のみ抽出
            df = df.rename(columns={2:"card",23:"新規入会数"})
            
            #ブランド分け
            df.loc[df['card'].str.contains("VISA"), 'brand'] = 'VISA'
            df.loc[df['card'].str.contains("Master"), 'brand'] = 'Master'
            df.loc[df['card'] .str.contains("臨時"), 'brand'] = 'VISA'
            df.loc[df['card'] .str.contains("ハウス"), 'brand'] = 'ハウス'

            df.loc[df['card'].str.contains("グループ※"), 'card'] = '一般'
            df.loc[df['card'].str.contains("臨時"), 'card'] = '臨時カード'
            df.loc[df['card'].str.contains("ハウス"), 'card'] = 'ハウスゴールド'
            
            df.loc[df['card'].str.match('VISA'), 'card'] = '一般'
            df.loc[df['card'].str.match('Master'), 'card'] = '一般'

            df['card'] = df['card'].str.replace("VISA",'')
            df['card'] = df['card'].str.replace("Master",'')
            
            #年月の列を作成
            df['filename'] = f            
            df['month'] = input_sheet_name[i]
            df['month'] = df['month'].astype(int)
                                  
            df['year'] = df['filename'].str[:4]
            df['year'] = df['year'].astype(int)
            
            df.loc[3<=df['month'] , 'year'] = df['year']-1 #ファイル名から持ってくると前年度が引き算されないので
                       
            df = df.drop(columns = {"filename"})
            
            li.append(df)
            
            #df_all = pd.concat(li, axis=0, ignore_index=True)

#df_all = df_all.reindex(columns = ['year','month','brand','card','新規入会数'])

Wall time: 625 ms


2018年度用

In [306]:
%%time
for f in old_filename:
    input_file = f
    input_book = pd.ExcelFile(data_dir/ f"{input_file}")
    #各シート名を取得
    input_sheet_name = input_book.sheet_names
    num_sheet = len(input_sheet_name)
    
    for i in range(0,num_sheet-3):#不要なシートが一つあるので読みこまない
        df = input_book.parse(input_sheet_name[i],header = None)
        
        if f[:8] == "2019年2月度":
            df = df.iloc[[4,5,6,7,8,9,11,12,15,16],[2,21]]#該当行と列のみ抽出
            df = df.rename(columns={2:"card",21:"新規入会数"})
            
            #ブランド分け
            df.loc[df['card'].str.contains("VISA"), 'brand'] = 'VISA'
            df.loc[df['card'].str.contains("Master"), 'brand'] = 'Master'
            df.loc[df['card'] .str.contains("臨時"), 'brand'] = 'VISA'
            df.loc[df['card'] .str.contains("ハウス"), 'brand'] = 'ハウス'

            df.loc[df['card'].str.contains("グループ※"), 'card'] = '一般'
            df.loc[df['card'].str.contains("臨時"), 'card'] = '臨時カード'
            df.loc[df['card'].str.contains("ハウス"), 'card'] = 'ハウスゴールド'
            
            df.loc[df['card'].str.match('VISA'), 'card'] = '一般'
            df.loc[df['card'].str.match('Master'), 'card'] = '一般'

            df['card'] = df['card'].str.replace("VISA",'')
            df['card'] = df['card'].str.replace("Master",'')
            
            #年月の列を作成
            df['filename'] = f            
            df['month'] = input_sheet_name[i]
            df['month'] = df['month'].astype(int)
                                  
            df['year'] = df['filename'].str[:4]
            df['year'] = df['year'].astype(int)
            
            df.loc[3<=df['month'] , 'year'] = df['year']-1 #ファイル名から持ってくると前年度が引き算されないので
                       
            df = df.drop(columns = {"filename"})
            
            li.append(df)
            
            df_all = pd.concat(li, axis=0, ignore_index=True)

df_all = df_all.reindex(columns = ['year','month','brand','card','新規入会数'])

Wall time: 682 ms


In [307]:
df_all = pd.concat(li, axis=0, ignore_index=True)
df_all = df_all.reindex(columns = ['year','month','brand','card','新規入会数'])
df_all = df_all.sort_values(['year','month','brand','card'])

In [308]:
df_all.head()

,year,month,brand,card,新規入会数
185,2018,3,Master,ゴールド,56
183,2018,3,Master,一般,1149
191,2018,3,Master,一般,2
189,2018,3,Master,外商,311
184,2018,3,VISA,ゴールド,206


In [309]:
df_all.groupby(['year','month'])['新規入会数'].sum()

year  month
2018  3        11386
      4        11410
      5        14040
      6        10703
      7         9902
      8        10369
      9         9066
      10       10840
      11       12602
      12       12401
2019  1         8826
      2         9062
      4         6441
      5         8331
      6         8977
      7        10715
      8         6731
      9         7995
      10        6361
      11        8366
      12        9559
2020  1        10003
      2         7632
      3         5531
      4         1405
      5         1456
      6         2967
      7         4193
      8         3839
      9         3953
      10        7504
Name: 新規入会数, dtype: int64

Excelアウトプット

In [311]:
df_all.to_excel(data_dir /r"Output"/f"【集計】承認口座数_{today_yyyymmdd}.xlsx"
                ,sheet_name ='raw_承認口座数',index=None,header=True,encoding='utf-8-sig')